In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [24]:
import pandas as pd
from datetime import datetime, timedelta
from sklearn import datasets
from scipy import stats
import numpy as np
from evidently.test_suite import TestSuite
# from from evidently.legacy.test_suite import TestSuite
from evidently.test_preset import DataDriftTestPreset, DataQualityTestPreset, DataStabilityTestPreset
# from evidently.legacy.test_preset import DataDriftTestPreset, DataQualityTestPreset, DataStabilityTestPreset

In [38]:
df = pd.read_csv("../data/adult.csv")
ref_data = df[1:25000]
curr_data = df[25000:32561]

In [26]:
df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [27]:
df.select_dtypes(include = 'int').columns

Index(['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss',
       'hours.per.week'],
      dtype='object')

In [28]:
df.select_dtypes(include = ['object']).columns

Index(['workclass', 'education', 'marital.status', 'occupation',
       'relationship', 'race', 'sex', 'native.country', 'income'],
      dtype='object')

In [29]:
numerical_columns = [
   'age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss',
       'hours.per.week'
]

# drop non-numerical columns
df = df.drop(['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country', 'income'], axis=1)

z_threshold = 3
z_scores = np.abs(stats.zscore(df[numerical_columns]))
outliers = np.where(z_scores > z_threshold)
df_no_outliers = df[(z_scores <= z_threshold).all(axis=1)]

In [44]:
data_drift_suite = TestSuite(tests=[DataDriftTestPreset()])
data_drift_suite.run(reference_data=ref_data, current_data=curr_data)

if not data_drift_suite.as_dict()['summary']['all_passed']:
    data_drift_suite.save_html("../notebooks/monitoring/data_drift_suite.html")

c:\Users\Sawitt\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning:

invalid value encountered in divide

c:\Users\Sawitt\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning:

invalid value encountered in divide



In [46]:
model_quality_suite = TestSuite(tests=[DataQualityTestPreset(), DataStabilityTestPreset()])
reference_data = df[1:25000]
current_data = df[25000:32561]
model_quality_suite.run(reference_data=ref_data, current_data=curr_data)

if not model_quality_suite.as_dict()['summary']['all_passed']:
    model_quality_suite.save_html("../notebooks/monitoring/model_quality_suite.html")